In [1]:
import numpy as np
import pickle, os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from datetime import datetime
import time
import sys
import os

sys.path.append(os.path.abspath("..")) # Adds the parent folder to sys.path
from utils import *
from model import *
from Neural_Nets import *

/home/x_lab/workspace/l2o/SSLMIQP_main/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/x_lab/workspace/l2o/SSLMIQP_main/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

In [ ]:
relative_path = os.getcwd()
relative_path = os.path.abspath("..")
dataset_fn = relative_path + '/self_supervised' + '/single.p'
prob_features = ['x0', 'xg']

data_file = open(dataset_fn,'rb')
all_data = pickle.load(data_file)[:100000]  # use only part of the dataset for quick testing
data_file.close()
num_train = len(all_data)
print(f"Number of training samples: {num_train}")

X0 = np.vstack([all_data[ii]['x0'].T for ii in range(num_train)])  
XG = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
OBS = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
XX = np.array([all_data[ii]['XX'] for ii in range(num_train)])
UU = np.array([all_data[ii]['UU'] for ii in range(num_train)])
YY = np.concatenate([all_data[ii]['YY'].astype(int) for ii in range(num_train)], axis=1).transpose(1,0,2)
train_data = [{'x0': X0, 'xg': XG}, {'XX': XX, 'UU' : UU}, YY]
Obs_info = np.array([[1.0, 0.0, 0.4, 0.5, 0.0],
        [0.7, -1.1, 0.5, 0.4, 0.0],
        [0.40, -2.50, 0.4, 0.5, 0.0]])


Number of training samples: 939136


# Phase 1: Regression with Feed-forward NNs

In [3]:
# Build the FFNet model
FFNet_reg = Regression(prob_features)
n_features = 6 # the dimension of feature (input vector)
FFNet_reg.setup_data(n_features, train_data, Obs_info)

In [4]:
# FFNet_reg.Y_train.shape
# FFNet_reg.P_train["XX"].shape
# FFNet_reg.n_obs


In [5]:
FFNet_reg.setup_network(depth=4, neurons=1024)
FFNet_reg.model

training_params = {}
training_params['TRAINING_EPOCHS'] = int(1000)
training_params['BATCH_SIZE'] = 200
training_params['CHECKPOINT_AFTER'] = int(1)
training_params['SAVEPOINT_AFTER'] = int(10)
training_params['TEST_BATCH_SIZE'] = 100
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['EARLY_STOPPING_PATIENCE'] = 5

FFNet_reg.train(training_params, verbose=True)

wandb: Currently logged in as: mux2001 (mux2001-xxxxxlab-test1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Epoch 0], [Iter 4228] Saved model at /home/x_lab/workspace/l2o/SSLMIQP_main/self_supervised/regression_20251017_1717.pt
[Epoch 0], [Iter 4228] Validation loss: 0.4649 | Validation accuracy (bitwise): 0.9822 | Constraint violation: 0.0323
[Epoch 1], [Iter 8455] Validation loss: 0.4619 | Validation accuracy (bitwise): 0.9898 | Constraint violation: 0.0230
[Epoch 2], [Iter 12682] Validation loss: 0.4623 | Validation accuracy (bitwise): 0.9897 | Constraint violation: 0.0334
[Epoch 3], [Iter 16909] Validation loss: 0.4620 | Validation accuracy (bitwise): 0.9901 | Constraint violation: 0.0262
[Epoch 4], [Iter 21136] Validation loss: 0.4621 | Validation accuracy (bitwise): 0.9896 | Constraint violation: 0.0258
[Epoch 5], [Iter 25363] Validation loss: 0.4624 | Validation accuracy (bitwise): 0.9895 | Constraint violation: 0.0281


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[Epoch 6], [Iter 29590] Validation loss: 0.4621 | Validation accuracy (bitwise): 0.9897 | Constraint violation: 0.0230
Early stopping: no improvement for 5 epochs
Final model saved at /home/x_lab/workspace/l2o/SSLMIQP_main/self_supervised/regression_20251017_1717.pt


avg_train_loss,█▂▂▁▁▁▁
epoch,▁▁▂▂▃▃▅▅▆▆▇▇██
iteration,▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████
train_loss,█▇▇▇▇▅▇▅▄▆▄▅▄▃▂▃▃▃▂▃▂▃▂▂▃▂▂▁▄▂▂▁▃▄▄▂▂▂▂▂
val/bitwise_acc,▁████▇█
val/constraint_violation,▇▁█▃▃▄▁
val/loss,█▁▂▁▁▂▁
avg_train_loss,0.46056
epoch,6
iteration,29589
train_loss,0.46228


# Phase 2: Refine with Self-Supervised Learning

In [6]:
fn_saved = 'regression_20251007_1650.pt'  
FFNet_reg.setup_network(depth=4, neurons=1024)
FFNet_reg.load_network(fn_saved)
print(FFNet_reg.model)

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=1024, bias=True)
    (1-3): 3 x Linear(in_features=1024, out_features=1024, bias=True)
    (4): Linear(in_features=1024, out_features=240, bias=True)
  )
)


In [7]:
training_params = {}
training_params['TRAINING_EPOCHS'] = int(1000)
training_params['BATCH_SIZE'] = 200
training_params['CHECKPOINT_AFTER'] = int(1)
training_params['SAVEPOINT_AFTER'] = int(10)
training_params['TEST_BATCH_SIZE'] = 100
training_params['LEARNING_RATE'] = 1e-5
training_params['WEIGHT_DECAY'] = 1e-4
training_params['EARLY_STOPPING_PATIENCE'] = 5

FFNet_reg.SS_train(training_params, verbose=True, penalty_weight=1.0)


[Epoch 0], [Iter 4228] Saved model at /home/x_lab/workspace/l2o/SSLMIQP_main/self_supervised/regression_20251017_1718.pt
[Epoch 0], [Iter 4228] Validation loss: 0.8066 | Validation accuracy (bitwise): 0.6917 | Constraint violation: 0.2320
[Epoch 1], [Iter 8455] Validation loss: 0.8979 | Validation accuracy (bitwise): 0.6936 | Constraint violation: 0.3240
[Epoch 2], [Iter 12682] Validation loss: 0.6645 | Validation accuracy (bitwise): 0.6992 | Constraint violation: 0.0927
[Epoch 3], [Iter 16909] Validation loss: 0.6907 | Validation accuracy (bitwise): 0.6959 | Constraint violation: 0.1177
[Epoch 4], [Iter 21136] Validation loss: 0.6516 | Validation accuracy (bitwise): 0.6993 | Constraint violation: 0.0799
[Epoch 5], [Iter 25363] Validation loss: 0.6486 | Validation accuracy (bitwise): 0.6978 | Constraint violation: 0.0763
[Epoch 6], [Iter 29590] Validation loss: 0.7032 | Validation accuracy (bitwise): 0.6909 | Constraint violation: 0.1283
[Epoch 7], [Iter 33817] Validation loss: 0.6649 

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[Epoch 10], [Iter 46498] Validation loss: 0.6786 | Validation accuracy (bitwise): 0.6927 | Constraint violation: 0.1044
Early stopping: no improvement for 5 epochs
Final model saved at /home/x_lab/workspace/l2o/SSLMIQP_main/self_supervised/regression_20251017_1718.pt


avg_train_loss,█▂▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇██
iteration,▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▁▂▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/bitwise_acc,▂▄█▅█▇▂▃▃▁▃
val/constraint_violation,▅█▁▂▁▁▂▁▃▂▂
val/loss,▅█▁▂▁▁▃▁▃▂▂
avg_train_loss,0.11439
epoch,10
iteration,46497
train_loss,0.09091
